In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi
from mplsoccer import Pitch
import matplotlib as mpl
import psycopg2
import os
import dotenv

# Connect to the database
dotenv.load_dotenv()
conn = psycopg2.connect(
    host=os.getenv("PG_HOST"),
    database=os.getenv("PG_DB"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD"),
    port=os.getenv("PG_PORT"),
    sslmode="require"
)

# Fetch data directly
game_id = '5xud4wolwm30f88iddht6tnh0'

# Get tracking data for a specific game
query = """
SELECT pt.frame_id, pt.timestamp, pt.player_id, pt.x, pt.y, p.jersey_number, p.player_name, p.team_id
FROM player_tracking pt
JOIN players p ON pt.player_id = p.player_id
WHERE pt.game_id = %s
LIMIT 1000  -- Limit to avoid memory issues
"""

tracking_df = pd.read_sql_query(query, conn, params=[game_id])

# Function to plot Voronoi diagram
def plot_voronoi(tracking_data, frame_id):
    
    frame_data = tracking_data[tracking_data['frame_id'] == frame_id]
    frame_id = 1723406756000
    
    # Skip ball data for Voronoi calculation (assuming player_id 0 or similar for ball)
    players_df = frame_data[frame_data['player_name'] != 'Ball'].copy()
    
    # Get unique teams
    teams = players_df['team_id'].unique()
    team_colors = {teams[0]: 'red', teams[1]: 'blue'} if len(teams) > 1 else {teams[0]: 'red'}
    
    # Extract player coordinates
    points = players_df[['x', 'y']].values
    
    # Create pitch
    pitch = Pitch(pitch_color='grass', line_color='white', pitch_type='opta',
                  pitch_length=105, pitch_width=68)
    fig, ax = pitch.draw(figsize=(12, 8))
    
    # Calculate Voronoi diagram if we have enough points
    if len(points) >= 3:  
        
        boundary_points = np.array([
            [-10, -10], [115, -10], [115, 78], [-10, 78]  # Points outside the pitch
        ])
        vor_points = np.vstack([points, boundary_points])
        
        # Compute Voronoi
        vor = Voronoi(vor_points)
        
        # Plot Voronoi regions for players (not boundary points)
        for i in range(len(players_df)):
            region_idx = vor.point_region[i]
            if region_idx != -1:  # Valid region
                region = vor.regions[region_idx]
                if not -1 in region and len(region) > 0:  # No point at infinity
                    polygon = [vor.vertices[v] for v in region]
                    team_id = players_df.iloc[i]['team_id']
                    ax.fill(*zip(*polygon), alpha=0.4, color=team_colors[team_id])
    
    # Plot player positions
    for _, player in players_df.iterrows():
        x, y = player['x'], player['y']
        team_id = player['team_id']
        jersey_no = player['jersey_number']
        plt.scatter(x, y, s=100, color=team_colors[team_id], edgecolor='black', zorder=10)
        plt.text(x+1, y+1, str(jersey_no), fontsize=8, color='white', fontweight='bold', zorder=11)
    
    # Plot the ball if present
    ball_data = frame_data[frame_data['player_name'] == 'Ball']
    if not ball_data.empty:
        ball_x = ball_data['x'].values[0]
        ball_y = ball_data['y'].values[0]
        plt.scatter(ball_x, ball_y, s=80, color='yellow', edgecolor='black', zorder=12)
    
    timestamp = frame_data['timestamp'].iloc[0] if not frame_data.empty else "Unknown"
    plt.title(f"Voronoi Diagram - Frame: {frame_id}, Time: {timestamp}")
    
    return fig

# Select a frame to visualize
if not tracking_df.empty:
    unique_frames = tracking_df['frame_id'].unique()
    if len(unique_frames) > 0:
        selected_frame = unique_frames[0]  # Use first frame
        fig = plot_voronoi(tracking_df, selected_frame)
        plt.show()
    else:
        print("No frames found in the tracking data.")
else:
    print("No tracking data available for the specified game.")

# Close the connection
conn.close()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch
import matplotlib as mpl
from helperfunctions import get_database_connection, fetch_tracking_data

def visualize_football_field(game_id, frame_id=None, timestamp=None):
    """
    Visualize the football field with player positions for a specific frame or timestamp.
    
    Parameters:
    - game_id (str): ID of the game to visualize
    - frame_id (int/str): Optional specific frame ID to visualize
    - timestamp (str): Optional timestamp to visualize (format: 'MM:SS' or 'HH:MM:SS')
    """
    # Connect to the database
    conn = get_database_connection()
    
    try:
        # Fetch tracking data for the specified game
        tracking_df = fetch_tracking_data(game_id, conn)
        print(f"Fetched {len(tracking_df)} tracking data points")
        
        # Check if we have any data
        if tracking_df.empty:
            print(f"No tracking data available for game ID: {game_id}")
            return
        
        # Fetch team names to match with team IDs
        team_names_query = """
            SELECT team_id, team_name FROM teams 
            WHERE team_id IN (
                SELECT home_team_id FROM matches WHERE match_id = %s
                UNION
                SELECT away_team_id FROM matches WHERE match_id = %s
            )
            """
        team_names_df = pd.read_sql_query(team_names_query, conn, params=[game_id, game_id])
        
        # Create a mapping from team_id to team_name
        team_name_map = dict(zip(team_names_df['team_id'], team_names_df['team_name']))
        
        # Rest of your function remains the same until the team colors section
        
        # Convert timestamp to appropriate format if provided and frame_id is not
        if timestamp and not frame_id:
            # Add hours if missing
            if len(timestamp.split(':')) == 2:
                timestamp = f"00:{timestamp}"
                
            # Find the closest frame to the specified timestamp
            tracking_df['timestamp_dt'] = pd.to_timedelta(tracking_df['timestamp'])
            target_dt = pd.to_timedelta(timestamp)
            
            # Find the closest frame
            closest_idx = abs(tracking_df['timestamp_dt'] - target_dt).idxmin()
            frame_id = tracking_df.loc[closest_idx, 'frame_id']
            
            print(f"Selected frame {frame_id} closest to timestamp {timestamp}")
        
        # If no frame or timestamp specified, list available frames
        if not frame_id:
            # Show available frames
            frame_samples = sorted(tracking_df['frame_id'].unique())
            if len(frame_samples) > 10:
                print("First 5 available frame IDs:")
                for i, frame in enumerate(frame_samples[:5]):
                    ts = tracking_df[tracking_df['frame_id'] == frame]['timestamp'].iloc[0]
                    print(f"{i+1}. Frame ID: {frame}, Timestamp: {ts}")
                print("...")
                print("Last 5 available frame IDs:")
                for i, frame in enumerate(frame_samples[-5:]):
                    ts = tracking_df[tracking_df['frame_id'] == frame]['timestamp'].iloc[0]
                    print(f"{len(frame_samples)-4+i}. Frame ID: {frame}, Timestamp: {ts}")
            else:
                print("All available frame IDs:")
                for i, frame in enumerate(frame_samples):
                    ts = tracking_df[tracking_df['frame_id'] == frame]['timestamp'].iloc[0]
                    print(f"{i+1}. Frame ID: {frame}, Timestamp: {ts}")
            
            # Use the first frame if none specified
            frame_id = tracking_df['frame_id'].min()
            print(f"\nNo frame specified. Using first frame: {frame_id}")
        
        # Filter data for the specific frame
        frame_data = tracking_df[tracking_df['frame_id'] == frame_id]
        
        # Check if we have data for the requested frame
        if frame_data.empty:
            print(f"No data found for frame {frame_id}")
            return
            
        # Get the timestamp for the selected frame
        selected_timestamp = frame_data['timestamp'].iloc[0] if not frame_data.empty else "Unknown"
        
        # Create a pitch
        pitch = Pitch(pitch_color='grass', line_color='white', pitch_type='opta',
                    pitch_length=105, pitch_width=68)
        fig, ax = pitch.draw(figsize=(12, 8))
        
        # Find unique teams and assign colors
        teams = frame_data['team_id'].unique()
        team_colors = {}
        
        # Assign colors to teams (excluding the ball if present)
        team_colors_list = list(mpl.colors.TABLEAU_COLORS.values())
        i = 0
        
        for team in teams:
            # Check if this is not the ball (assuming ball might have a specific team_id)
            team_data = frame_data[frame_data['team_id'] == team]
            if not ('Ball' in team_data['player_name'].values):
                team_colors[team] = team_colors_list[i % len(team_colors_list)]
                i += 1
        
        # Plot player positions (same as before)
        for _, player in frame_data.iterrows():
            x = player['x']
            y = player['y']
            jersey_no = player['jersey_number']
            team_id = player['team_id']
            player_name = player['player_name']
            
            if player_name == 'Ball':
                # Plot the ball
                ax.scatter(x, y, s=100, color='yellow', edgecolor='black', zorder=20)
            else:
                # Plot the player
                color = team_colors.get(team_id, 'gray')  # Use gray for unassigned teams
                ax.scatter(x, y, s=120, color=color, edgecolor='black', zorder=10)
                
                # Add jersey number
                ax.text(x, y, str(jersey_no), fontsize=8, ha='center', va='center', 
                        color='white', fontweight='bold', zorder=11)
                
                # Add player name with small offset
                ax.text(x, y-2, player_name, fontsize=7, ha='center', color='white',
                        bbox=dict(facecolor='black', alpha=0.7, pad=1), zorder=12)
        
        # Add title with frame information
        ax.set_title(f"Player Positions - Game: {game_id}\nFrame: {frame_id}, Time: {selected_timestamp}", 
                    fontsize=14)
        
        # UPDATED: Add legend with team names instead of IDs
        handles = []
        labels = []
        for team_id, color in team_colors.items():
            # Use team name from the mapping if available, otherwise use team ID
            team_name = team_name_map.get(team_id, f"Team {team_id}")
            handles.append(plt.Line2D([0], [0], marker='o', color='w', 
                                     markerfacecolor=color, markersize=10))
            labels.append(team_name)
        
        # Add ball to legend if present
        ball_data = frame_data[frame_data['player_name'] == 'Ball']
        if not ball_data.empty:
            handles.append(plt.Line2D([0], [0], marker='o', color='w', 
                                    markerfacecolor='yellow', markersize=10))
            labels.append("Ball")
        
        if handles and labels:
            ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.05), 
                    fancybox=True, shadow=True, ncol=len(handles))
        
        plt.tight_layout()
        plt.show()
        
        return fig, ax
    
    finally:
        # Close the connection
        if conn:
            conn.close()
            print("Database connection closed.")

# erDiagram
#     Match {
#         string match_id PK
#         datetime match_date
#         string home_team_id FK
#         string away_team_id FK
#         int home_score
#         int away_score
#     }
    
#     Team {
#         string team_id PK
#         string team_name
#     }
    
#     Player {
#         string player_id PK
#         string player_name
#         string team_id FK
#         int jersey_number
#     }
    
#     PlayerTracking {
#         int id PK
#         string game_id FK
#         bigint frame_id
#         string timestamp
#         int period_id
#         string player_id FK
#         float x
#         float y
#     }
    
#     EventType {
#         string eventtype_id PK
#         string name
#         string description
#     }
    
#     QualifierType {
#         string qualifier_id PK
#         string name
#         string description
#     }
    
#     MatchEvent {
#         string match_id PK, FK
#         string event_id PK
#         string eventtype_id FK
#         string result
#         boolean success
#         int period_id
#         string timestamp
#         string end_timestamp
#         string ball_state
#         string ball_owning_team
#         string team_id FK
#         string player_id FK
#         float x
#         float y
#         float end_coordinates_x
#         float end_coordinates_y
#         string receiver_player_id FK
#     }
    
#     Qualifier {
#         string match_id PK, FK
#         string event_id PK, FK
#         string qualifier_type_id PK, FK
#         string qualifier_value
#     }
    
#     SpadlAction {
#         int id PK
#         string game_id FK
#         int period_id
#         float seconds
#         string player_id FK
#         string team_id FK
#         float start_x
#         float start_y
#         float end_x
#         float end_y
#         string action_type
#         string result
#         string bodypart
#     }
    
#     Match ||--o{ PlayerTracking : "has"
#     Match ||--o{ MatchEvent : "has"
#     Match ||--o{ SpadlAction : "has"
#     Team ||--o{ Player : "has"
#     Team ||--o{ MatchEvent : "participates in"
#     Team ||--o{ SpadlAction : "performs"
#     Player ||--o{ PlayerTracking : "has"
#     Player ||--o{ MatchEvent : "performs"
#     Player ||--o{ MatchEvent : "receives"
#     Player ||--o{ SpadlAction : "performs"
#     EventType ||--o{ MatchEvent : "categorizes"
#     MatchEvent ||--o{ Qualifier : "has"
#     QualifierType ||--o{ Qualifier : "categorizes"
#     Team ||--o{ Match : "home team"
#     Team ||--o{ Match : "away team"

# Example usage - use one of these patterns:

# 1. Get information about available frames for a game
game_id = '5uts2s7fl98clqz8uymaazehg'
# visualize_football_field(game_id)

# 2. Visualize a specific frame
# visualize_football_field(game_id, frame_id=1722805622280)

# 3. Visualize at a specific timestamp
visualize_football_field(game_id, timestamp="00:16:35")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch
import matplotlib as mpl
from scipy.spatial import Voronoi, voronoi_plot_2d
from helperfunctions import get_database_connection, fetch_tracking_data

def visualize_football_field(game_id, frame_id=None, timestamp=None, show_voronoi=True):
    """
    Visualize the football field with player positions and Voronoi diagrams for a specific frame or timestamp.
    
    Parameters:
    - game_id (str): ID of the game to visualize
    - frame_id (int/str): Optional specific frame ID to visualize
    - timestamp (str): Optional timestamp to visualize (format: 'MM:SS' or 'HH:MM:SS')
    - show_voronoi (bool): Whether to display Voronoi diagrams (default: True)
    """
    # Connect to the database
    conn = get_database_connection()
    
    try:
        # Fetch tracking data for the specified game
        tracking_df = fetch_tracking_data(game_id, conn)
        print(f"Fetched {len(tracking_df)} tracking data points")
        
        # Check if we have any data
        if tracking_df.empty:
            print(f"No tracking data available for game ID: {game_id}")
            return
        
        # Fetch team names to match with team IDs
        team_names_query = """
        SELECT team_id, team_name FROM teams
        WHERE team_id IN (
            SELECT DISTINCT team_id FROM players
            WHERE player_id IN (
                SELECT DISTINCT player_id FROM player_tracking
                WHERE game_id = %s
            )
        )
        """
        team_names_df = pd.read_sql_query(team_names_query, conn, params=[game_id])
        
        # Create a mapping from team_id to team_name
        team_name_map = dict(zip(team_names_df['team_id'], team_names_df['team_name']))
        
        # Convert timestamp to appropriate format if provided and frame_id is not
        if timestamp and not frame_id:
            # Add hours if missing
            if len(timestamp.split(':')) == 2:
                timestamp = f"00:{timestamp}"
                
            # Find the closest frame to the specified timestamp
            tracking_df['timestamp_dt'] = pd.to_timedelta(tracking_df['timestamp'])
            target_dt = pd.to_timedelta(timestamp)
            
            # Find the closest frame
            closest_idx = abs(tracking_df['timestamp_dt'] - target_dt).idxmin()
            frame_id = tracking_df.loc[closest_idx, 'frame_id']
            
            print(f"Selected frame {frame_id} closest to timestamp {timestamp}")
        
        # If no frame or timestamp specified, list available frames
        if not frame_id:
            # Show available frames
            frame_samples = sorted(tracking_df['frame_id'].unique())
            if len(frame_samples) > 10:
                print("First 5 available frame IDs:")
                for i, frame in enumerate(frame_samples[:5]):
                    ts = tracking_df[tracking_df['frame_id'] == frame]['timestamp'].iloc[0]
                    print(f"{i+1}. Frame ID: {frame}, Timestamp: {ts}")
                print("...")
                print("Last 5 available frame IDs:")
                for i, frame in enumerate(frame_samples[-5:]):
                    ts = tracking_df[tracking_df['frame_id'] == frame]['timestamp'].iloc[0]
                    print(f"{len(frame_samples)-4+i}. Frame ID: {frame}, Timestamp: {ts}")
            else:
                print("All available frame IDs:")
                for i, frame in enumerate(frame_samples):
                    ts = tracking_df[tracking_df['frame_id'] == frame]['timestamp'].iloc[0]
                    print(f"{i+1}. Frame ID: {frame}, Timestamp: {ts}")
            
            # Use the first frame if none specified
            frame_id = tracking_df['frame_id'].min()
            print(f"\nNo frame specified. Using first frame: {frame_id}")
        
        # Filter data for the specific frame
        frame_data = tracking_df[tracking_df['frame_id'] == frame_id]
        
        # Check if we have data for the requested frame
        if frame_data.empty:
            print(f"No data found for frame {frame_id}")
            return
            
        # Get the timestamp for the selected frame
        selected_timestamp = frame_data['timestamp'].iloc[0] if not frame_data.empty else "Unknown"
        
        # Create a pitch
        pitch = Pitch(pitch_color='grass', line_color='white', pitch_type='opta',
                    pitch_length=105, pitch_width=68)
        fig, ax = pitch.draw(figsize=(12, 8))
        
        # Find unique teams and assign colors
        teams = frame_data['team_id'].unique()
        team_colors = {}
        
        # Assign colors to teams (excluding the ball if present)
        team_colors_list = list(mpl.colors.TABLEAU_COLORS.values())
        i = 0
        
        for team in teams:
            # Check if this is not the ball (assuming ball might have a specific team_id)
            team_data = frame_data[frame_data['team_id'] == team]
            if not ('Ball' in team_data['player_name'].values):
                team_colors[team] = team_colors_list[i % len(team_colors_list)]
                i += 1
                
        # Generate Voronoi diagram if requested
        if show_voronoi:
            # Filter out the ball and extract only player positions
            players_only = frame_data[frame_data['player_name'] != 'Ball']
            if len(players_only) >= 3:  # Need at least 3 points for a meaningful Voronoi diagram
                # Extract player positions
                points = players_only[['x', 'y']].values
                
                # Create Voronoi diagram
                try:
                    # Add boundary points to ensure Voronoi regions are contained within the pitch
                    pitch_boundary = np.array([
                        [0, 0], [105, 0], [105, 68], [0, 68],  # Corners
                        [0, 34], [105, 34],  # Middle of sidelines
                        [52.5, 0], [52.5, 68]  # Middle of goal lines
                    ])
                    
                    # Combine player positions with boundary points
                    all_points = np.vstack([points, pitch_boundary])
                    
                    # Calculate Voronoi diagram
                    vor = Voronoi(all_points)
                    
                    # Plot Voronoi diagram for players only (not boundary points)
                    for i, (_, player) in enumerate(players_only.iterrows()):
                        region_idx = vor.point_region[i]
                        region = vor.regions[region_idx]
                        
                        # Only plot finite regions
                        if -1 not in region and len(region) > 0:
                            polygon = [vor.vertices[j] for j in region]
                            polygon_array = np.array(polygon)
                            
                            # Get team color
                            team_id = player['team_id']
                            color = team_colors.get(team_id, 'gray')
                            
                            # Plot the Voronoi cell with transparency
                            ax.fill(polygon_array[:, 0], polygon_array[:, 1], 
                                   alpha=0.3, color=color, edgecolor='gray',
                                   linewidth=1, zorder=5)
                except Exception as e:
                    print(f"Error generating Voronoi diagram: {e}")
                    # Continue without Voronoi if it fails
        
        # Plot player positions
        for _, player in frame_data.iterrows():
            x = player['x']
            y = player['y']
            jersey_no = player['jersey_number']
            team_id = player['team_id']
            player_name = player['player_name']
            
            if player_name == 'Ball':
                # Plot the ball
                ax.scatter(x, y, s=100, color='yellow', edgecolor='black', zorder=20)
            else:
                # Plot the player
                color = team_colors.get(team_id, 'gray')  # Use gray for unassigned teams
                ax.scatter(x, y, s=120, color=color, edgecolor='black', zorder=10)
                
                # Add jersey number
                ax.text(x, y, str(jersey_no), fontsize=8, ha='center', va='center', 
                        color='white', fontweight='bold', zorder=11)
                
                # Add player name with small offset
                ax.text(x, y-2, player_name, fontsize=7, ha='center', color='white',
                        bbox=dict(facecolor='black', alpha=0.7, pad=1), zorder=12)
        
        # Add title with frame information
        title = f"Player Positions - Game: {game_id}\nFrame: {frame_id}, Time: {selected_timestamp}"
        if show_voronoi:
            title += " (with Voronoi Diagram)"
        ax.set_title(title, fontsize=14)
        
        # Add legend with team names
        handles = []
        labels = []
        for team_id, color in team_colors.items():
            # Use team name from the mapping if available, otherwise use team ID
            team_name = team_name_map.get(team_id, f"Team {team_id}")
            handles.append(plt.Line2D([0], [0], marker='o', color='w', 
                                     markerfacecolor=color, markersize=10))
            labels.append(team_name)
        
        # Add ball to legend if present
        ball_data = frame_data[frame_data['player_name'] == 'Ball']
        if not ball_data.empty:
            handles.append(plt.Line2D([0], [0], marker='o', color='w', 
                                    markerfacecolor='yellow', markersize=10))
            labels.append("Ball")
        
        if handles and labels:
            ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.05), 
                    fancybox=True, shadow=True, ncol=len(handles))
        
        plt.tight_layout()
        plt.show()
        
        return fig, ax
    
    finally:
        # Close the connection
        if conn:
            conn.close()
            print("Database connection closed.")

# Example usage - use one of these patterns:

# 1. Get information about available frames for a game
game_id = '5uts2s7fl98clqz8uymaazehg'
# visualize_football_field(game_id)

# 2. Visualize a specific frame
# visualize_football_field(game_id, frame_id=1722805622280)

# 3. Visualize at a specific timestamp with Voronoi diagram
visualize_football_field(game_id, timestamp="00:16:35", show_voronoi=True)

# 4. Visualize without Voronoi diagram
# visualize_football_field(game_id, timestamp="00:16:35", show_voronoi=False)

In [ ]:
pip install shapely

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch
import matplotlib as mpl
import matplotlib.patches as patches
from helperfunctions import get_database_connection, fetch_tracking_data

def visualize_football_field_with_actions(game_id, frame_id=None, timestamp=None, show_actions=True, action_window=15):
    """
    Visualize the football field with player positions and SPADL actions.
    
    Parameters:
    - game_id (str): ID of the game to visualize
    - frame_id (int/str): Optional specific frame ID to visualize
    - timestamp (str): Optional timestamp to visualize (format: 'MM:SS' or 'HH:MM:SS')
    - show_actions (bool): Whether to show SPADL actions
    - action_window (int): Time window in seconds to show actions before and after the timestamp
    """
    # Connect to the database
    conn = get_database_connection()
    
    try:
        # Fetch tracking data for the specified game
        tracking_df = fetch_tracking_data(game_id, conn)
        print(f"Fetched {len(tracking_df)} tracking data points")
        
        # Check if we have any data
        if tracking_df.empty:
            print(f"No tracking data available for game ID: {game_id}")
            return
        
        # Fetch team names to match with team IDs
        team_names_query = """
        SELECT team_id, team_name FROM teams
        WHERE team_id IN (
            SELECT DISTINCT team_id FROM players
            WHERE player_id IN (
                SELECT DISTINCT player_id FROM player_tracking
                WHERE game_id = %s
            )
        )
        """
        team_names_df = pd.read_sql_query(team_names_query, conn, params=[game_id])
        team_name_map = dict(zip(team_names_df['team_id'], team_names_df['team_name']))
        
        # Convert timestamp to appropriate format if provided and frame_id is not
        if timestamp and not frame_id:
            # Add hours if missing
            if len(timestamp.split(':')) == 2:
                timestamp = f"00:{timestamp}"
                
            # Find the closest frame to the specified timestamp
            tracking_df['timestamp_dt'] = pd.to_timedelta(tracking_df['timestamp'])
            target_dt = pd.to_timedelta(timestamp)
            
            # Find the closest frame
            closest_idx = abs(tracking_df['timestamp_dt'] - target_dt).idxmin()
            frame_id = tracking_df.loc[closest_idx, 'frame_id']
            selected_timestamp = tracking_df.loc[closest_idx, 'timestamp']
            print(f"Selected frame {frame_id} closest to timestamp {timestamp}")
        else:
            # If frame ID is provided or no timestamp is provided, get timestamp from tracking data
            if frame_id:
                frame_data = tracking_df[tracking_df['frame_id'] == frame_id]
                if not frame_data.empty:
                    selected_timestamp = frame_data['timestamp'].iloc[0]
                else:
                    print(f"No data found for frame {frame_id}")
                    return
            else:
                # Use the first frame if none specified
                frame_id = tracking_df['frame_id'].min()
                selected_timestamp = tracking_df[tracking_df['frame_id'] == frame_id]['timestamp'].iloc[0]
                print(f"\nNo frame specified. Using first frame: {frame_id}")
        
        # Fetch SPADL actions around the selected timestamp
        if show_actions and selected_timestamp:
            # Convert the timestamp to seconds
            time_parts = selected_timestamp.split(':')
            if len(time_parts) == 3:
                selected_time_seconds = int(time_parts[0]) * 3600 + int(time_parts[1]) * 60 + int(time_parts[2])
            else:
                selected_time_seconds = int(time_parts[0]) * 60 + int(time_parts[1])
            
            # Calculate time window
            min_time = max(0, selected_time_seconds - action_window)
            max_time = selected_time_seconds + action_window
            
            # Fetch SPADL actions within the time window
            spadl_query = """
            SELECT sa.id, sa.game_id, sa.period_id, sa.seconds, sa.player_id, sa.team_id, 
                   sa.start_x, sa.start_y, sa.end_x, sa.end_y, sa.action_type, sa.result, sa.bodypart,
                   p.player_name, t.team_name
            FROM spadl_actions sa
            JOIN players p ON sa.player_id = p.player_id
            JOIN teams t ON sa.team_id = t.team_id
            WHERE sa.game_id = %s AND sa.seconds BETWEEN %s AND %s
            ORDER BY sa.seconds ASC
            """
            spadl_df = pd.read_sql_query(spadl_query, conn, params=[game_id, min_time, max_time])
            print(f"Fetched {len(spadl_df)} SPADL actions within ±{action_window} seconds window")
        else:
            spadl_df = pd.DataFrame()
        
        # Filter tracking data for the specific frame
        frame_data = tracking_df[tracking_df['frame_id'] == frame_id]
        
        # Check if we have data for the requested frame
        if frame_data.empty:
            print(f"No data found for frame {frame_id}")
            return
        
        # Create a pitch
        pitch = Pitch(pitch_color='grass', line_color='white', pitch_type='opta',
                    pitch_length=105, pitch_width=68)
        fig, ax = pitch.draw(figsize=(14, 10))
        
        # Find unique teams and assign colors
        teams = frame_data['team_id'].unique()
        team_colors = {}
        
        # Assign colors to teams (excluding the ball if present)
        team_colors_list = list(mpl.colors.TABLEAU_COLORS.values())
        i = 0
        
        for team in teams:
            team_data = frame_data[frame_data['team_id'] == team]
            if not ('Ball' in team_data['player_name'].values):
                team_colors[team] = team_colors_list[i % len(team_colors_list)]
                i += 1
        
        # Plot player positions
        for _, player in frame_data.iterrows():
            x = player['x']
            y = player['y']
            jersey_no = player['jersey_number']
            team_id = player['team_id']
            player_name = player['player_name']
            
            if player_name == 'Ball':
                # Plot the ball
                ax.scatter(x, y, s=100, color='yellow', edgecolor='black', zorder=20)
            else:
                # Plot the player
                color = team_colors.get(team_id, 'gray')  # Use gray for unassigned teams
                ax.scatter(x, y, s=120, color=color, edgecolor='black', zorder=10)
                
                # Add jersey number
                ax.text(x, y, str(jersey_no), fontsize=8, ha='center', va='center', 
                        color='white', fontweight='bold', zorder=11)
                
                # Add player name with small offset
                ax.text(x, y-2, player_name, fontsize=7, ha='center', color='white',
                        bbox=dict(facecolor='black', alpha=0.7, pad=1), zorder=12)
        
        # Plot SPADL actions if available
        if not spadl_df.empty:
            # Define action colors and markers
            action_colors = {
                'pass': 'lime',
                'cross': 'cyan',
                'shot': 'red',
                'dribble': 'orange',
                'tackle': 'purple',
                'interception': 'magenta',
                'clearance': 'gray',
                'foul': 'black',
                'save': 'pink'
            }
            
            # Define arrow properties based on action success
            success_props = {'width': 2, 'head_width': 3, 'head_length': 3, 'fc': 'green', 'ec': 'green', 'alpha': 0.8, 'zorder': 5}
            fail_props = {'width': 2, 'head_width': 3, 'head_length': 3, 'fc': 'red', 'ec': 'red', 'alpha': 0.8, 'zorder': 5}
            
            # Plot each action
            for _, action in spadl_df.iterrows():
                start_x = action['start_x']
                start_y = action['start_y']
                end_x = action['end_x']
                end_y = action['end_y']
                action_type = action['action_type']
                result = action['result']
                player_name = action['player_name']
                seconds = action['seconds']
                team_id = action['team_id']
                team_name = team_name_map.get(team_id, f"Team {team_id}")
                
                # Set color based on action type
                color = action_colors.get(action_type.lower(), 'gray')
                
                # Draw arrow for the action
                arrow_props = success_props if result == 'success' else fail_props
                ax.arrow(start_x, start_y, end_x - start_x, end_y - start_y, **arrow_props)
                
                # Add action marker at start position
                ax.scatter(start_x, start_y, s=80, color=color, marker='o', 
                          edgecolor='black', alpha=0.7, zorder=9)
                
                # Add action label with time and type
                time_min = int(seconds // 60)
                time_sec = int(seconds % 60)
                ax.text(start_x, start_y + 3, f"{time_min}:{time_sec:02d} - {action_type}",
                       fontsize=7, ha='center', va='bottom',
                       bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.3'), zorder=15)
        
        # Add title with frame information
        title = f"Player Positions - Game: {game_id}\n"
        title += f"Frame: {frame_id}, Time: {selected_timestamp}"
        if not spadl_df.empty:
            title += f"\nShowing {len(spadl_df)} actions within ±{action_window} seconds"
        ax.set_title(title, fontsize=14)
        
        # Add legend for teams and actions
        handles = []
        labels = []
        
        # Team legend
        for team_id, color in team_colors.items():
            team_name = team_name_map.get(team_id, f"Team {team_id}")
            handles.append(plt.Line2D([0], [0], marker='o', color='w', 
                                    markerfacecolor=color, markersize=10))
            labels.append(team_name)
        
        # Ball legend
        ball_data = frame_data[frame_data['player_name'] == 'Ball']
        if not ball_data.empty:
            handles.append(plt.Line2D([0], [0], marker='o', color='w', 
                                    markerfacecolor='yellow', markersize=10))
            labels.append("Ball")
        
        # Action legend
        if not spadl_df.empty:
            handles.append(plt.Line2D([0], [0], color='green', lw=2))
            labels.append("Successful Action")
            
            handles.append(plt.Line2D([0], [0], color='red', lw=2))
            labels.append("Failed Action")
            
            # Add specific action types if they exist in the data
            for action_type, color in action_colors.items():
                if action_type in spadl_df['action_type'].str.lower().values:
                    handles.append(plt.Line2D([0], [0], marker='o', color='w',
                                            markerfacecolor=color, markersize=8))
                    labels.append(action_type.capitalize())
        
        # Add the legend
        if handles and labels:
            ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.05), 
                    fancybox=True, shadow=True, ncol=min(5, len(handles)))
        
        plt.tight_layout()
        plt.show()
        
        return fig, ax
    
    finally:
        # Close the connection
        if conn:
            conn.close()
            print("Database connection closed.")

# Example usage
game_id = '5uts2s7fl98clqz8uymaazehg'

# 1. Visualize with default settings (first frame, with actions)
# visualize_football_field_with_actions(game_id)

# 2. Visualize a specific frame with actions
# visualize_football_field_with_actions(game_id, frame_id=1722805622280, show_actions=True)

# 3. Visualize at a specific timestamp with actions
visualize_football_field_with_actions(game_id, timestamp="00:16:35", action_window=10)

# 4. Visualize without showing actions
# visualize_football_field_with_actions(game_id, timestamp="00:16:35", show_actions=False)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch
import matplotlib as mpl
from scipy.spatial import Voronoi, voronoi_plot_2d
from matplotlib.patches import Polygon
from shapely.geometry import Polygon as ShapelyPolygon
from helperfunctions import get_database_connection, fetch_tracking_data

def visualize_football_field(game_id, frame_id=None, timestamp=None, show_voronoi=True, debug_player=None):
    """
    Visualize the football field with player positions and Voronoi diagrams for a specific frame or timestamp.
    Also calculates and displays the percentage of field controlled by each team.
    
    Parameters:
    - game_id (str): ID of the game to visualize
    - frame_id (int/str): Optional specific frame ID to visualize
    - timestamp (str): Optional timestamp to visualize (format: 'MM:SS' or 'HH:MM:SS')
    - show_voronoi (bool): Whether to display Voronoi diagrams (default: True)
    - debug_player (str): Optional player name to debug their Voronoi cell
    """
    # Connect to the database
    conn = get_database_connection()
    
    try:
        # Fetch tracking data for the specified game
        tracking_df = fetch_tracking_data(game_id, conn)
        print(f"Fetched {len(tracking_df)} tracking data points")
        
        # Check if we have any data
        if tracking_df.empty:
            print(f"No tracking data available for game ID: {game_id}")
            return
        
        # Fetch team names to match with team IDs
        team_names_query = """
        SELECT team_id, team_name FROM teams
        WHERE team_id IN (
            SELECT DISTINCT team_id FROM players
            WHERE player_id IN (
                SELECT DISTINCT player_id FROM player_tracking
                WHERE game_id = %s
            )
        )
        """
        team_names_df = pd.read_sql_query(team_names_query, conn, params=[game_id])
        
        # Create a mapping from team_id to team_name
        team_name_map = dict(zip(team_names_df['team_id'], team_names_df['team_name']))
        
        # Convert timestamp to appropriate format if provided and frame_id is not
        if timestamp and not frame_id:
            # Add hours if missing
            if len(timestamp.split(':')) == 2:
                timestamp = f"00:{timestamp}"
                
            # Find the closest frame to the specified timestamp
            tracking_df['timestamp_dt'] = pd.to_timedelta(tracking_df['timestamp'])
            target_dt = pd.to_timedelta(timestamp)
            
            # Find the closest frame
            closest_idx = abs(tracking_df['timestamp_dt'] - target_dt).idxmin()
            frame_id = tracking_df.loc[closest_idx, 'frame_id']
            
            print(f"Selected frame {frame_id} closest to timestamp {timestamp}")
        
        # If no frame or timestamp specified, list available frames
        if not frame_id:
            # Show available frames
            frame_samples = sorted(tracking_df['frame_id'].unique())
            if len(frame_samples) > 10:
                print("First 5 available frame IDs:")
                for i, frame in enumerate(frame_samples[:5]):
                    ts = tracking_df[tracking_df['frame_id'] == frame]['timestamp'].iloc[0]
                    print(f"{i+1}. Frame ID: {frame}, Timestamp: {ts}")
                print("...")
                print("Last 5 available frame IDs:")
                for i, frame in enumerate(frame_samples[-5:]):
                    ts = tracking_df[tracking_df['frame_id'] == frame]['timestamp'].iloc[0]
                    print(f"{len(frame_samples)-4+i}. Frame ID: {frame}, Timestamp: {ts}")
            else:
                print("All available frame IDs:")
                for i, frame in enumerate(frame_samples):
                    ts = tracking_df[tracking_df['frame_id'] == frame]['timestamp'].iloc[0]
                    print(f"{i+1}. Frame ID: {frame}, Timestamp: {ts}")
            
            # Use the first frame if none specified
            frame_id = tracking_df['frame_id'].min()
            print(f"\nNo frame specified. Using first frame: {frame_id}")
        
        # Filter data for the specific frame
        frame_data = tracking_df[tracking_df['frame_id'] == frame_id]
        
        # Check if we have data for the requested frame
        if frame_data.empty:
            print(f"No data found for frame {frame_id}")
            return
            
        # Get the timestamp for the selected frame
        selected_timestamp = frame_data['timestamp'].iloc[0] if not frame_data.empty else "Unknown"
        
        # Create a pitch
        pitch = Pitch(pitch_color='grass', line_color='white', pitch_type='opta',
                    pitch_length=105, pitch_width=68)
        fig, ax = pitch.draw(figsize=(12, 8))
        
        # Find unique teams and assign colors
        teams = frame_data['team_id'].unique()
        team_colors = {}
        
        # Assign colors to teams (excluding the ball if present)
        team_colors_list = list(mpl.colors.TABLEAU_COLORS.values())
        i = 0
        
        for team in teams:
            # Check if this is not the ball (assuming ball might have a specific team_id)
            team_data = frame_data[frame_data['team_id'] == team]
            if not ('Ball' in team_data['player_name'].values):
                team_colors[team] = team_colors_list[i % len(team_colors_list)]
                i += 1
        
        # Create variables to store team control data
        team_control = {team_id: 0 for team_id in team_colors.keys()}
        total_pitch_area = 105 * 68  # Total area of the pitch
        voronoi_polygons = []  # Store Voronoi polygons for area calculation
        
        # Create variable to store debug player info
        debug_player_info = None
        debug_idx = None
                
        # Generate Voronoi diagram if requested
        if show_voronoi:
            # Filter out the ball and extract only player positions
            players_only = frame_data[frame_data['player_name'] != 'Ball']
            
            # Find debug player if requested
            if debug_player:
                debug_player_rows = players_only[players_only['player_name'].str.contains(debug_player, case=False)]
                if not debug_player_rows.empty:
                    debug_idx = debug_player_rows.index[0]
                    debug_player_info = players_only.loc[debug_idx].to_dict()
                    print(f"\nDebug info for {debug_player}:")
                    print(f"Position: ({debug_player_info['x']}, {debug_player_info['y']})")
                    print(f"Team ID: {debug_player_info['team_id']}")
                    print(f"Jersey: {debug_player_info['jersey_number']}")
                    print(f"Player name: {debug_player_info['player_name']}")

            if len(players_only) >= 3:  # Need at least 3 points for a meaningful Voronoi diagram
                # Extract player positions
                points = players_only[['x', 'y']].values
                
                # Create Voronoi diagram
                try:
                    # Add boundary points to ensure Voronoi regions are contained within the pitch
                    pitch_boundary = np.array([
                        [0, 0], [105, 0], [105, 68], [0, 68],  # Corners
                        [0, 34], [105, 34],  # Middle of sidelines
                        [52.5, 0], [52.5, 68]  # Middle of goal lines
                    ])
                    
                    # Define the pitch polygon for clipping
                    pitch_polygon = ShapelyPolygon([
                        (0, 0), (105, 0), (105, 68), (0, 68)
                    ])
                    
                    # Combine player positions with boundary points
                    all_points = np.vstack([points, pitch_boundary])
                    
                    # Calculate Voronoi diagram
                    vor = Voronoi(all_points)
                    
                    # Debug player's Voronoi cell if requested
                    if debug_player_info is not None:
                        # Get the index of the player in the points array (before boundary points)
                        debug_point_idx = players_only.index.get_loc(debug_idx)
                        debug_region_idx = vor.point_region[debug_point_idx]
                        debug_region = vor.regions[debug_region_idx]
                        
                        print(f"\nVoronoi cell for {debug_player}:")
                        print(f"Region index: {debug_region_idx}")
                        print(f"Region vertices indices: {debug_region}")
                        print(f"Contains -1 (infinite): {-1 in debug_region}")
                    
                    # Plot Voronoi diagram for players only (not boundary points)
                    for i, (idx, player) in enumerate(players_only.iterrows()):
                        region_idx = vor.point_region[i]
                        region = vor.regions[region_idx]
                        
                        # Special handling for the debug player
                        is_debug_player = debug_player_info is not None and idx == debug_idx
                        
                        if is_debug_player:
                            print(f"Processing debug player at index {i}")
                            print(f"Region: {region}")
                        
                        # Handle both finite and infinite regions for the debug player
                        if is_debug_player and -1 in region:
                            print("Debug player has an infinite Voronoi region.")
                            print("Clipping it against the pitch boundaries...")
                            
                            # We'll try to construct a large polygon for this player
                            # First find the finite part of the Voronoi region
                            finite_part = [j for j in region if j != -1]
                            if finite_part:
                                points_base = [vor.vertices[j] for j in finite_part]
                                
                                # Extend to pitch boundaries in relevant directions
                                # This is a simplified approach - for better results, 
                                # you'd need to determine which edge of the Voronoi diagram 
                                # extends to infinity
                                extended_points = points_base + [
                                    (0, 0), (105, 0), (105, 68), (0, 68)
                                ]
                                try:
                                    hull_poly = ShapelyPolygon(extended_points)
                                    clipped_hull = hull_poly.intersection(pitch_polygon)
                                    
                                    # Plot this approximation with a different style
                                    if not clipped_hull.is_empty:
                                        if hasattr(clipped_hull, 'exterior'):
                                            x, y = clipped_hull.exterior.xy
                                            ax.fill(x, y, alpha=0.5, color='red', 
                                                  edgecolor='black', linewidth=2, zorder=7)
                                            print("Successfully created approximated region for debug player")
                                except Exception as ex:
                                    print(f"Error creating extended polygon: {ex}")
                                    
                        # Process finite regions normally
                        elif -1 not in region and len(region) > 0:
                            polygon = [vor.vertices[j] for j in region]
                            polygon_array = np.array(polygon)
                            
                            # Get team color
                            team_id = player['team_id']
                            color = team_colors.get(team_id, 'gray')
                            
                            # Use special styling for debug player
                            if is_debug_player:
                                color = 'red'
                                edgecolor = 'black'
                                alpha = 0.5
                                linewidth = 2
                                zorder = 7
                            else:
                                edgecolor = 'gray'
                                alpha = 0.3
                                linewidth = 1
                                zorder = 5
                            
                            # Plot the Voronoi cell
                            ax.fill(polygon_array[:, 0], polygon_array[:, 1], 
                                   alpha=alpha, color=color, edgecolor=edgecolor,
                                   linewidth=linewidth, zorder=zorder)
                            
                            # Calculate the area of the Voronoi cell within the pitch
                            voronoi_poly = ShapelyPolygon(polygon)
                            
                            # Clip the Voronoi polygon to stay within pitch boundaries
                            clipped_poly = voronoi_poly.intersection(pitch_polygon)
                            
                            # Print details for debug player
                            if is_debug_player:
                                print(f"Original polygon area: {voronoi_poly.area:.2f}")
                                print(f"Clipped polygon area: {clipped_poly.area:.2f}")
                                print(f"Is empty after clipping: {clipped_poly.is_empty}")
                            
                            # Store the polygon with team ID for later analysis
                            if not clipped_poly.is_empty:
                                voronoi_polygons.append({
                                    'team_id': team_id,
                                    'polygon': clipped_poly,
                                    'area': clipped_poly.area
                                })
                    
                    # Calculate team control percentages
                    for poly_data in voronoi_polygons:
                        team_control[poly_data['team_id']] += poly_data['area']
                    
                    # Convert to percentages
                    total_calculated_area = sum(team_control.values())
                    for team_id in team_control:
                        team_control[team_id] = (team_control[team_id] / total_calculated_area) * 100
                    
                except Exception as e:
                    print(f"Error generating Voronoi diagram: {e}")
                    import traceback
                    traceback.print_exc()
                    # Continue without Voronoi if it fails
        
        # Plot player positions
        for _, player in frame_data.iterrows():
            x = player['x']
            y = player['y']
            jersey_no = player['jersey_number']
            team_id = player['team_id']
            player_name = player['player_name']
            
            if player_name == 'Ball':
                # Plot the ball
                ax.scatter(x, y, s=100, color='yellow', edgecolor='black', zorder=20)
            else:
                # Special handling for debug player
                is_debug_player = debug_player and debug_player.lower() in player_name.lower()
                
                # Plot the player
                color = team_colors.get(team_id, 'gray')  # Use gray for unassigned teams
                if is_debug_player:
                    edgecolor = 'red'
                    linewidth = 2
                else:
                    edgecolor = 'black'
                    linewidth = 1
                    
                ax.scatter(x, y, s=120, color=color, edgecolor=edgecolor, 
                          linewidth=linewidth, zorder=10)
                
                # Add jersey number
                ax.text(x, y, str(jersey_no), fontsize=8, ha='center', va='center', 
                        color='white', fontweight='bold', zorder=11)
                
                # Add player name with small offset
                ax.text(x, y-2, player_name, fontsize=7, ha='center', color='white',
                        bbox=dict(facecolor='black' if not is_debug_player else 'red', 
                                  alpha=0.7, pad=1), zorder=12)
        
        # Add title with frame information
        title = f"Player Positions - Game: {game_id}\nFrame: {frame_id}, Time: {selected_timestamp}"
        if show_voronoi:
            title += " (with Voronoi Diagram)"
        ax.set_title(title, fontsize=14)
        
        # Add legend with team names and control percentages
        handles = []
        labels = []
        for team_id, color in team_colors.items():
            # Use team name from the mapping if available, otherwise use team ID
            team_name = team_name_map.get(team_id, f"Team {team_id}")
            
            # Add control percentage if available
            if team_id in team_control:
                control_pct = team_control[team_id]
                team_label = f"{team_name} - {control_pct:.1f}% control"
            else:
                team_label = team_name
                
            handles.append(plt.Line2D([0], [0], marker='o', color='w', 
                                     markerfacecolor=color, markersize=10))
            labels.append(team_label)
        
        # Add ball to legend if present
        ball_data = frame_data[frame_data['player_name'] == 'Ball']
        if not ball_data.empty:
            handles.append(plt.Line2D([0], [0], marker='o', color='w', 
                                    markerfacecolor='yellow', markersize=10))
            labels.append("Ball")
        
        if handles and labels:
            ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.05), 
                    fancybox=True, shadow=True, ncol=len(handles))
        
        # Add team control percentages text box
        if show_voronoi and team_control:
            control_text = "Team Field Control:\n"
            for team_id, percentage in team_control.items():
                team_name = team_name_map.get(team_id, f"Team {team_id}")
                control_text += f"{team_name}: {percentage:.1f}%\n"
                
            # Position text box in top-right corner
            props = dict(boxstyle='round', facecolor='wheat', alpha=0.7)
            ax.text(0.97, 0.97, control_text, transform=ax.transAxes, fontsize=10,
                   verticalalignment='top', horizontalalignment='right', bbox=props)
            
            # Create control percentage bar chart at the bottom
            ax_bar = fig.add_axes([0.15, 0.02, 0.7, 0.05])  # [left, bottom, width, height]
            
            # Sort teams by control percentage
            teams_sorted = sorted(team_control.items(), key=lambda x: x[1], reverse=True)
            team_ids = [team_id for team_id, _ in teams_sorted]
            percentages = [pct for _, pct in teams_sorted]
            colors = [team_colors[team_id] for team_id in team_ids]
            
            # Create stacked horizontal bar chart
            left = 0
            for i, (team_id, pct) in enumerate(zip(team_ids, percentages)):
                ax_bar.barh(0, pct, left=left, color=colors[i], height=0.8)
                # Add percentage text if space allows
                if pct > 5:
                    ax_bar.text(left + pct/2, 0, f"{pct:.1f}%", 
                            ha='center', va='center', color='white', fontweight='bold')
                left += pct
            
            # Remove axes ticks and labels
            ax_bar.set_yticks([])
            ax_bar.set_xticks([])
            ax_bar.set_xlim(0, 100)
            ax_bar.spines['top'].set_visible(False)
            ax_bar.spines['right'].set_visible(False)
            ax_bar.spines['bottom'].set_visible(False)
            ax_bar.spines['left'].set_visible(False)
            ax_bar.set_title("Team Field Control", fontsize=10, pad=2)
        
        plt.tight_layout()
        plt.show()
        
        return fig, ax, team_control if show_voronoi else (fig, ax, None)
    
    finally:
        # Close the connection
        if conn:
            conn.close()
            print("Database connection closed.")

# Example usage - use one of these patterns:

# 1. Get information about available frames for a game
game_id = '5uts2s7fl98clqz8uymaazehg'
# visualize_football_field(game_id)

# 2. Visualize a specific frame
# visualize_football_field(game_id, frame_id=1722805622280)

# 3. Visualize at a specific timestamp with Voronoi diagram and control percentages
# Add debug_player parameter to investigate A. Brown's Voronoi cell
fig, ax, team_control = visualize_football_field(game_id, timestamp="00:16:35", show_voronoi=True, debug_player="Brown")

# Print detailed control percentages
if team_control:
    print("\nDetailed Field Control Analysis:")
    for team_id, percentage in team_control.items():
        print(f"Team {team_id}: {percentage:.2f}%")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import animation
from mplsoccer import Pitch
import matplotlib as mpl
import matplotlib.patches as patches
from helperfunctions import get_database_connection, fetch_tracking_data
from IPython.display import HTML

# Increase animation size limit
mpl.rcParams['animation.embed_limit'] = 50

def create_football_animation(game_id, start_timestamp, end_timestamp, fps=10):
    """
    Create an animation of player movements for a specific time range.
    SPADL actions functionality is commented out but preserved for future use.
    
    Parameters:
    - game_id (str): ID of the game to visualize
    - start_timestamp (str): Starting timestamp (format: 'MM:SS' or 'HH:MM:SS')
    - end_timestamp (str): Ending timestamp (format: 'MM:SS' or 'HH:MM:SS')
    - fps (int): Frames per second for the animation
    
    Returns:
    - animation: The created animation object
    """
    # Connect to the database
    conn = get_database_connection()
    
    try:
        # Add hours to timestamps if missing
        if len(start_timestamp.split(':')) == 2:
            start_timestamp = f"00:{start_timestamp}"
        if len(end_timestamp.split(':')) == 2:
            end_timestamp = f"00:{end_timestamp}"
        
        # Convert to timedelta for comparison
        start_td = pd.to_timedelta(start_timestamp)
        end_td = pd.to_timedelta(end_timestamp)
        
        # Fetch tracking data for the specified game
        tracking_df = fetch_tracking_data(game_id, conn)
        print(f"Fetched {len(tracking_df)} tracking data points")
        
        # Convert timestamps to timedelta
        tracking_df['timestamp_dt'] = pd.to_timedelta(tracking_df['timestamp'])
        
        # Filter data for the specified time range
        mask = (tracking_df['timestamp_dt'] >= start_td) & (tracking_df['timestamp_dt'] <= end_td)
        filtered_tracking = tracking_df[mask]
        
        print(f"Selected {len(filtered_tracking)} tracking points between {start_timestamp} and {end_timestamp}")
        
        # Get unique frames to animate
        unique_frames = filtered_tracking['frame_id'].unique()
        frame_timestamps = {
            frame: filtered_tracking[filtered_tracking['frame_id'] == frame]['timestamp'].iloc[0]
            for frame in unique_frames
        }
        
        print(f"Found {len(unique_frames)} unique frames to animate")
        
        # Sort frames chronologically
        unique_frames = sorted(unique_frames)
        
        # Fetch team names
        team_names_query = """
        SELECT team_id, team_name FROM teams
        WHERE team_id IN (
            SELECT DISTINCT team_id FROM players
            WHERE player_id IN (
                SELECT DISTINCT player_id FROM player_tracking
                WHERE game_id = %s
            )
        )
        """
        team_names_df = pd.read_sql_query(team_names_query, conn, params=[game_id])
        team_name_map = dict(zip(team_names_df['team_id'], team_names_df['team_name']))
        
        # # COMMENTED OUT: SPADL actions functionality
        # # Convert timestamps to seconds for SPADL actions
        # start_td_seconds = int(start_td.total_seconds())
        # end_td_seconds = int(end_td.total_seconds())
        # 
        # # Fetch all SPADL actions for the time range (plus action_window padding)
        # spadl_query = """
        # SELECT sa.id, sa.game_id, sa.period_id, sa.seconds, sa.player_id, sa.team_id, 
        #        sa.start_x, sa.start_y, sa.end_x, sa.end_y, sa.action_type, sa.result, sa.bodypart,
        #        p.player_name, t.team_name
        # FROM spadl_actions sa
        # JOIN players p ON sa.player_id = p.player_id
        # JOIN teams t ON sa.team_id = t.team_id
        # WHERE sa.game_id = %s AND sa.seconds BETWEEN %s AND %s
        # ORDER BY sa.seconds ASC
        # """
        # all_actions = pd.read_sql_query(spadl_query, conn, params=[
        #     game_id,
        #     max(0, start_td_seconds - action_window),
        #     end_td_seconds + action_window
        # ])
        # 
        # print(f"Fetched {len(all_actions)} SPADL actions for the time range")
        
        # Set up the figure and pitch
        pitch = Pitch(pitch_color='grass', line_color='white', pitch_type='opta',
                      pitch_length=105, pitch_width=68)
        fig, ax = pitch.draw(figsize=(14, 10))
        
        # # COMMENTED OUT: Action colors and markers definition
        # # Define action colors and markers
        # action_colors = {
        #     'pass': 'lime',
        #     'cross': 'cyan',
        #     'shot': 'red',
        #     'dribble': 'orange',
        #     'tackle': 'purple',
        #     'interception': 'magenta', 
        #     'clearance': 'gray',
        #     'foul': 'black',
        #     'save': 'pink'
        # }
        # 
        # # Define arrow properties based on action success
        # success_props = {'width': 2, 'head_width': 3, 'head_length': 3, 'fc': 'green', 'ec': 'green', 'alpha': 0.8, 'zorder': 5}
        # fail_props = {'width': 2, 'head_width': 3, 'head_length': 3, 'fc': 'red', 'ec': 'red', 'alpha': 0.8, 'zorder': 5}
        
        # Find unique teams and assign colors
        teams = filtered_tracking['team_id'].unique()
        team_colors = {}
        
        # Assign colors to teams (excluding the ball if present)
        team_colors_list = list(mpl.colors.TABLEAU_COLORS.values())
        i = 0
        
        for team in teams:
            team_data = filtered_tracking[filtered_tracking['team_id'] == team]
            if not ('Ball' in team_data['player_name'].values):
                team_colors[team] = team_colors_list[i % len(team_colors_list)]
                i += 1
        
        # Initialize player dots, labels and action elements
        player_dots = {}
        player_labels = {}
        player_names = {}
        # action_elements = []  # Commented out - not needed without SPADL
        
        # Create title
        title = ax.set_title("", fontsize=14)
        
        # Create legend for teams and ball
        legend_elements = []
        for team_id, color in team_colors.items():
            team_name = team_name_map.get(team_id, f"Team {team_id}")
            legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', 
                                           markerfacecolor=color, markersize=10, label=team_name))
        
        # Add ball to legend
        legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', 
                                        markerfacecolor='yellow', markersize=10, label="Ball"))
        
        # # COMMENTED OUT: Action legend elements
        # # Add action legend elements
        # legend_elements.append(plt.Line2D([0], [0], color='green', lw=2, label="Successful Action"))
        # legend_elements.append(plt.Line2D([0], [0], color='red', lw=2, label="Failed Action"))
        
        # Add the legend
        legend = ax.legend(handles=legend_elements, 
                          loc='upper center', 
                          bbox_to_anchor=(0.5, -0.05),
                          fancybox=True, 
                          shadow=True, 
                          ncol=min(6, len(legend_elements)))
        
        def init():
            """Initialize the animation"""
            title.set_text("")
            # # COMMENTED OUT: Clear action elements
            # for element in action_elements:
            #     if isinstance(element, plt.Line2D):
            #         element.set_data([], [])
            #     else:
            #         element.remove()
            # 
            # action_elements.clear()
            return [title, legend] + list(player_dots.values()) + list(player_labels.values()) + list(player_names.values())
        
        def animate(frame_idx):
            """Animate one frame"""
            frame_id = unique_frames[frame_idx]
            frame_timestamp = frame_timestamps[frame_id]
            
            # # COMMENTED OUT: Clear previous actions
            # for element in action_elements:
            #     if isinstance(element, plt.Line2D):
            #         element.set_data([], [])
            #     elif element in ax.texts:
            #         element.remove()
            #     elif element in ax.collections:
            #         element.remove()
            #     elif element in ax.patches:
            #         element.remove()
            # 
            # action_elements.clear()
            
            # Update title
            title.set_text(f"Game: {game_id} - Time: {frame_timestamp}")
            
            # Get current frame data
            frame_data = filtered_tracking[filtered_tracking['frame_id'] == frame_id]
            
            # # Convert the timestamp to seconds for action window
            # time_parts = frame_timestamp.split(':')
            # try:
            #     if len(time_parts) == 3:
            #         # Handle hours:minutes:seconds format
            #         hours = int(time_parts[0])
            #         minutes = int(time_parts[1])
            #         # Handle decimal seconds by converting to float first, then to int
            #         seconds = int(float(time_parts[2]))
            #         curr_seconds = hours * 3600 + minutes * 60 + seconds
            #     else:
            #         # Handle minutes:seconds format
            #         minutes = int(time_parts[0])
            #         # Handle decimal seconds by converting to float first, then to int
            #         seconds = int(float(time_parts[1]))
            #         curr_seconds = minutes * 60 + seconds
            # except ValueError as e:
            #     print(f"Warning: Could not parse timestamp {frame_timestamp}: {e}")
            #     # Fall back to just using the frame index as proxy for time
            #     curr_seconds = frame_idx * 30  # Assuming 30 fps
            # 
            # # Find actions within the time window
            # min_time = max(0, curr_seconds - action_window)
            # max_time = curr_seconds + action_window
            # frame_actions = all_actions[(all_actions['seconds'] >= min_time) & 
            #                          (all_actions['seconds'] <= max_time)]
            
            # Update player positions
            for _, player in frame_data.iterrows():
                player_id = player['player_id']
                x, y = player['x'], player['y']
                jersey_no = player['jersey_number']
                team_id = player['team_id']
                player_name = player['player_name']
                
                # Create or update player dots
                if player_id not in player_dots:
                    if player_name == 'Ball':
                        player_dots[player_id] = ax.scatter(x, y, s=100, color='yellow', 
                                                         edgecolor='black', zorder=20)
                    else:
                        color = team_colors.get(team_id, 'gray')
                        player_dots[player_id] = ax.scatter(x, y, s=120, color=color, 
                                                         edgecolor='black', zorder=10)
                else:
                    player_dots[player_id].set_offsets([x, y])
                
                # Create or update jersey number labels
                if player_id not in player_labels and player_name != 'Ball':
                    player_labels[player_id] = ax.text(x, y, str(jersey_no), fontsize=8, 
                                                    ha='center', va='center',
                                                    color='white', fontweight='bold', zorder=11)
                elif player_name != 'Ball':
                    player_labels[player_id].set_position((x, y))
                
                # Create or update player name labels
                if player_id not in player_names and player_name != 'Ball':
                    player_names[player_id] = ax.text(x, y-2, player_name, fontsize=7, 
                                                   ha='center', color='white',
                                                   bbox=dict(facecolor='black', alpha=0.7, pad=1), 
                                                   zorder=12)
                elif player_name != 'Ball':
                    player_names[player_id].set_position((x, y-2))
            # # Draw actions
            # for _, action in frame_actions.iterrows():
            #     start_x = action['start_x']
            #     start_y = action['start_y']
            #     end_x = action['end_x']
            #     end_y = action['end_y']
            #     action_type = action['action_type']
            #     result = action['result']
            #     seconds = action['seconds']
            #     
            #     # Set color based on action type
            #     color = action_colors.get(action_type.lower(), 'gray')
            #     
            #     # Draw arrow for the action
            #     arrow_props = success_props if result == 'success' else fail_props
            #     
            #     # Calculate the direction vector
            #     dx, dy = end_x - start_x, end_y - start_y
            #     
            #     # Only draw arrows if there's a meaningful displacement
            #     if abs(dx) > 1 or abs(dy) > 1:  # Threshold to avoid tiny arrows
            #         action_arrow = ax.arrow(start_x, start_y, dx, dy, **arrow_props)
            #         action_elements.append(action_arrow)
            #     
            #     # Add action marker at start position
            #     action_marker = ax.scatter(start_x, start_y, s=80, color=color, marker='o',
            #                              edgecolor='black', alpha=0.7, zorder=9)
            #     action_elements.append(action_marker)
            #     
            #     # Add action label with time and type
            #     time_min = int(seconds // 60)
            #     time_sec = int(seconds % 60)
            #     action_label = ax.text(start_x, start_y + 3, f"{time_min}:{time_sec:02d} - {action_type}",
            #                         fontsize=7, ha='center', va='bottom',
            #                         bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.3'), 
            #                         zorder=15)
            #     action_elements.append(action_label)
            
            return [title, legend] + list(player_dots.values()) + list(player_labels.values()) + list(player_names.values()) # + action_elements
        
        # Create animation
        if len(unique_frames) > 0:
            print(f"Creating animation with {len(unique_frames)} frames at {fps} fps...")
            anim = animation.FuncAnimation(
                fig, animate, init_func=init,
                frames=len(unique_frames), interval=1000/fps, blit=True
            )
            
            print("Animation created successfully!")
            return anim
        else:
            print("No frames found for the specified time range.")
            return None
    
    finally:
        # Close the connection
        if conn:
            conn.close()
            print("Database connection closed.")


# Example usage:
game_id = '5oc8drrbruovbuiriyhdyiyok'
start_time = "00:00:00"
end_time = "00:00:30"

anim = create_football_animation(game_id, start_time, end_time, fps=1)

# Save animation (optional)
# anim.save('football_animation.mp4', writer='ffmpeg', fps=10, dpi=300)

# Display animation in notebook
from IPython.display import HTML
HTML(anim.to_jshtml())

In [ ]:
# Does not work

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import animation
from mplsoccer import Pitch
import matplotlib as mpl
import matplotlib.patches as patches
from scipy.spatial import Voronoi
from shapely.geometry import Polygon as ShapelyPolygon, box
import warnings
from matplotlib.collections import PolyCollection
from helperfunctions import get_database_connection, fetch_tracking_data
from IPython.display import HTML

# Suppress specific shapely warnings that might occur during processing
warnings.filterwarnings("ignore", category=UserWarning, module="shapely")

# Increase animation size limit
mpl.rcParams['animation.embed_limit'] = 110

def create_football_animation(game_id, start_timestamp, end_timestamp, fps=10, show_voronoi=True):
    """
    Create an animation of player movements with Voronoi diagrams for a specific time range.
    
    Parameters:
    - game_id (str): ID of the game to visualize
    - start_timestamp (str): Starting timestamp (format: 'MM:SS' or 'HH:MM:SS')
    - end_timestamp (str): Ending timestamp (format: 'MM:SS' or 'HH:MM:SS')
    - fps (int): Frames per second for the animation
    - show_voronoi (bool): Whether to display Voronoi diagrams
    
    Returns:
    - animation: The created animation object
    """
    # Connect to the database
    conn = get_database_connection()
    
    try:
        # Add hours to timestamps if missing
        if len(start_timestamp.split(':')) == 2:
            start_timestamp = f"00:{start_timestamp}"
        if len(end_timestamp.split(':')) == 2:
            end_timestamp = f"00:{end_timestamp}"
        
        # Convert to timedelta for comparison
        start_td = pd.to_timedelta(start_timestamp)
        end_td = pd.to_timedelta(end_timestamp)
        
        # Fetch tracking data for the specified game
        tracking_df = fetch_tracking_data(game_id, conn)
        print(f"Fetched {len(tracking_df)} tracking data points")
        
        # Convert timestamps to timedelta
        tracking_df['timestamp_dt'] = pd.to_timedelta(tracking_df['timestamp'])
        
        # Filter data for the specified time range
        mask = (tracking_df['timestamp_dt'] >= start_td) & (tracking_df['timestamp_dt'] <= end_td)
        filtered_tracking = tracking_df[mask]
        
        print(f"Selected {len(filtered_tracking)} tracking points between {start_timestamp} and {end_timestamp}")
        
        # Get unique frames to animate
        unique_frames = filtered_tracking['frame_id'].unique()
        frame_timestamps = {
            frame: filtered_tracking[filtered_tracking['frame_id'] == frame]['timestamp'].iloc[0]
            for frame in unique_frames
        }
        
        print(f"Found {len(unique_frames)} unique frames to animate")
        
        # Sort frames chronologically
        unique_frames = sorted(unique_frames)
        
        # Fetch team names
        team_names_query = """
        SELECT team_id, team_name FROM teams
        WHERE team_id IN (
            SELECT DISTINCT team_id FROM players
            WHERE player_id IN (
                SELECT DISTINCT player_id FROM player_tracking
                WHERE game_id = %s
            )
        )
        """
        team_names_df = pd.read_sql_query(team_names_query, conn, params=[game_id])
        team_name_map = dict(zip(team_names_df['team_id'], team_names_df['team_name']))
        
        # Find unique teams and assign colors (check in filtered_tracking data only)
        team_ids = []
        for _, player in filtered_tracking.iterrows():
            if player['player_name'] != 'Ball' and player['team_id'] not in team_ids:
                team_ids.append(player['team_id'])
        
        # Print found teams for debugging
        print(f"Found {len(team_ids)} teams in tracking data:")
        for team_id in team_ids:
            team_name = team_name_map.get(team_id, f"Unknown Team ({team_id})")
            print(f"  - {team_name} (ID: {team_id})")
        
        # Assign colors to teams
        team_colors = {}
        team_colors_list = list(mpl.colors.TABLEAU_COLORS.values())
        for i, team_id in enumerate(team_ids):
            team_colors[team_id] = team_colors_list[i % len(team_colors_list)]
        
        # Set up the figure and pitch
        pitch = Pitch(pitch_color='grass', line_color='white', pitch_type='opta',
                      pitch_length=105, pitch_width=68)
        fig = plt.figure(figsize=(14, 10))
        
        # Create main axes for the pitch
        ax_main = fig.add_axes([0.05, 0.05, 0.9, 0.8])
        pitch.draw(ax=ax_main)
        
        # Create axes for the control percentages bar
        ax_control = fig.add_axes([0.15, 0.02, 0.7, 0.025])
        
        # Initialize player dots, labels and control
        player_dots = {}
        player_labels = {}
        player_names = {}
        
        # Create title
        title = ax_main.set_title("", fontsize=14)
        
        # Create text box for team control percentages
        control_text = ax_main.text(0.97, 0.97, "", transform=ax_main.transAxes, fontsize=10,
                                   verticalalignment='top', horizontalalignment='right',
                                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))
        
        # Create legend for teams and ball
        legend_elements = []
        for team_id, color in team_colors.items():
            team_name = team_name_map.get(team_id, f"Team {team_id}")
            legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', 
                                           markerfacecolor=color, markersize=10, label=team_name))
        
        # Add ball to legend
        legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', 
                                        markerfacecolor='yellow', markersize=10, label="Ball"))
        
        # Add Voronoi indicator to legend if showing Voronoi diagrams
        if show_voronoi:
            for team_id, color in team_colors.items():
                team_name = team_name_map.get(team_id, f"Team {team_id}")
                legend_elements.append(patches.Patch(facecolor=color, edgecolor='black',
                                                  alpha=0.3, label=f"{team_name} Control"))
        
        # Add the legend
        legend = ax_main.legend(handles=legend_elements, 
                              loc='upper center', 
                              bbox_to_anchor=(0.5, -0.05),
                              fancybox=True, 
                              shadow=True, 
                              ncol=min(4, len(legend_elements)))
        
        # Create a collection for Voronoi cells that we'll update
        voronoi_collection = None
        
        def voronoi_finite_polygons_2d(vor, boundary):
            """
            Compute finite Voronoi polygons constrained by a boundary.
            
            Parameters:
            - vor: Voronoi tessellation object
            - boundary: ShapelyPolygon representing the boundary (pitch)
            
            Returns:
            - list of (team_id, polygon) tuples
            """
            team_polygons = []
            
            # Get boundary as a shapely box
            boundary_polygon = boundary
            boundary_shape = boundary_polygon.convex_hull
            
            # Add far points to help with infinite regions
            center = vor.points.mean(axis=0)
            radius = 2 * np.sqrt(boundary_shape.area)  # Large radius to ensure coverage
            
            # For each Voronoi region...
            for i, (point, region_idx) in enumerate(zip(vor.points, vor.point_region)):
                team_id = team_ids_for_points[i]
                
                # Get the Voronoi region vertices
                region = vor.regions[region_idx]
                
                # Skip empty or infinite regions without processing
                if -1 in region or len(region) == 0:
                    # Instead of skipping, create a "far region" around the point
                    # This is a simplified approach that creates a large circle around the point
                    # and then clips it to the boundary
                    
                    # Create a temporary polygon that extends in all directions
                    temp_boundary = box(point[0]-radius, point[1]-radius, 
                                        point[0]+radius, point[1]+radius)
                    
                    # Create polygons for all the well-defined regions
                    all_valid_polygons = []
                    for j, reg in enumerate(vor.regions):
                        if -1 not in reg and len(reg) > 0:
                            poly_verts = [vor.vertices[v] for v in reg]
                            if len(poly_verts) >= 3:  # Need at least 3 points for a polygon
                                try:
                                    all_valid_polygons.append(ShapelyPolygon(poly_verts))
                                except Exception:
                                    pass  # Skip invalid polygons
                    
                    if all_valid_polygons:
                        # Combine all valid polygons
                        union_polygon = ShapelyPolygon(temp_boundary)
                        for poly in all_valid_polygons:
                            try:
                                # Subtract each valid polygon from our temporary boundary
                                if poly.is_valid:
                                    union_polygon = union_polygon.difference(poly)
                            except Exception:
                                pass  # Skip problematic polygons
                        
                        # Now find the part of this polygon that contains our current point
                        from shapely.ops import nearest_points
                        from shapely.geometry import Point, MultiPolygon
                        
                        our_point = Point(point[0], point[1])
                        
                        # If union_polygon is a MultiPolygon, find the part containing our point
                        if isinstance(union_polygon, MultiPolygon):
                            containing_poly = None
                            for part in union_polygon.geoms:
                                if part.contains(our_point) or part.distance(our_point) < 1:
                                    containing_poly = part
                                    break
                                
                            if containing_poly is not None:
                                clipped = containing_poly.intersection(boundary_shape)
                                if not clipped.is_empty:
                                    team_polygons.append((team_id, clipped))
                        else:
                            # Single polygon case
                            clipped = union_polygon.intersection(boundary_shape)
                            if not clipped.is_empty:
                                team_polygons.append((team_id, clipped))
                else:
                    # This is a normal, finite Voronoi region
                    polygon_vertices = [vor.vertices[i] for i in region]
                    if len(polygon_vertices) >= 3:  # Need at least 3 points for a polygon
                        try:
                            poly = ShapelyPolygon(polygon_vertices)
                            clipped = poly.intersection(boundary_shape)
                            if not clipped.is_empty:
                                team_polygons.append((team_id, clipped))
                        except Exception as e:
                            # Just log the error and continue
                            print(f"Error processing polygon: {e}")
            
            # Check if we're missing any areas
            all_polys = [poly for _, poly in team_polygons]
            if all_polys:
                try:
                    coverage = ShapelyPolygon()
                    for poly in all_polys:
                        coverage = coverage.union(poly)
                    remaining = boundary_shape.difference(coverage)
                    
                    if not remaining.is_empty and remaining.area > 0.01:
                        # There's uncovered area. Let's assign it based on nearest player
                        if hasattr(remaining, 'geoms'):
                            geoms = remaining.geoms
                        else:
                            geoms = [remaining]
                        
                        from shapely.geometry import Point
                        
                        for geom in geoms:
                            if geom.area > 0.01:  # Skip tiny fragments
                                centroid = geom.centroid
                                nearest_idx = np.argmin(
                                    [Point(pt[0], pt[1]).distance(centroid) for pt in vor.points]
                                )
                                nearest_team = team_ids_for_points[nearest_idx]
                                team_polygons.append((nearest_team, geom))
                except Exception as e:
                    print(f"Error checking coverage: {e}")
            
            return team_polygons
        
        def init():
            """Initialize the animation"""
            nonlocal voronoi_collection
            
            title.set_text("")
            control_text.set_text("")
            
            # Reset control bar
            ax_control.clear()
            ax_control.set_yticks([])
            ax_control.set_xticks([])
            
            if voronoi_collection is not None:
                voronoi_collection.remove()
                voronoi_collection = None
            
            # Return all artists that will be animated
            return [title, control_text, legend]
        
        def update_voronoi_diagram(frame_data):
            """Update Voronoi diagram for the current frame"""
            nonlocal voronoi_collection, team_ids_for_points
            
            # Remove previous Voronoi cells if they exist
            if voronoi_collection is not None:
                voronoi_collection.remove()
                voronoi_collection = None
            
            # Check if we have enough players for Voronoi
            players_only = frame_data[frame_data['player_name'] != 'Ball']
            if len(players_only) < 3:  # Need at least 3 points
                return {}
            
            # Extract player positions and team info for coloring
            points = []
            team_ids_for_points = []
            
            # Boundary of the pitch
            min_x, max_x = 0, 105
            min_y, max_y = 0, 68
            pitch_boundary = ShapelyPolygon([
                (min_x, min_y), (max_x, min_y), 
                (max_x, max_y), (min_x, max_y)
            ])
            
            # Add virtual points at the corners of the pitch to ensure coverage
            virtual_points = [
                [min_x-1, min_y-1],
                [max_x+1, min_y-1],
                [max_x+1, max_y+1],
                [min_x-1, max_y+1]
            ]
            
            # Define which team gets the out-of-bounds areas (alternating)
            virtual_team_pattern = [team_ids[0], team_ids[1], team_ids[0], team_ids[1]]
            
            # Add player positions (within field bounds)
            for _, player in players_only.iterrows():
                x, y = player['x'], player['y']
                # Allow a small margin outside the pitch for players
                if min_x-5 <= x <= max_x+5 and min_y-5 <= y <= max_y+5:
                    points.append([x, y])
                    team_ids_for_points.append(player['team_id'])
            
            # Add the virtual points if needed
            if len(points) < 3:
                # Not enough players, so rely on virtual points
                points.extend(virtual_points[:4-len(points)])
                team_ids_for_points.extend(virtual_team_pattern[:4-len(team_ids_for_points)])
            else:
                # Add at least one virtual point from each corner to ensure coverage
                points.extend(virtual_points)
                team_ids_for_points.extend(virtual_team_pattern)
            
            points = np.array(points)
            
            try:
                # Compute Voronoi diagram
                vor = Voronoi(points)
                
                # Get team polygons
                team_polygons = voronoi_finite_polygons_2d(vor, pitch_boundary)
                
                # Prepare data for polygon collection
                polygon_data = []
                polygon_colors = []
                
                # Calculate team control
                team_control = {team_id: 0 for team_id in team_colors}
                
                # Process the polygons
                for team_id, poly in team_polygons:
                    try:
                        # Add to team control calculation
                        area = poly.area
                        team_control[team_id] += area
                        
                        # Add polygon to visualization data
                        if hasattr(poly, 'exterior'):
                            # Single polygon
                            x, y = poly.exterior.xy
                            polygon_data.append(list(zip(x, y)))
                            polygon_colors.append(team_colors.get(team_id, 'gray'))
                        elif hasattr(poly, 'geoms'):
                            # MultiPolygon
                            for geom in poly.geoms:
                                if hasattr(geom, 'exterior'):
                                    x, y = geom.exterior.xy
                                    polygon_data.append(list(zip(x, y)))
                                    polygon_colors.append(team_colors.get(team_id, 'gray'))
                    except Exception as e:
                        print(f"Error processing polygon: {e}")
                
                # Create the polygon collection
                if polygon_data:
                    voronoi_collection = PolyCollection(
                        polygon_data, 
                        facecolors=[mpl.colors.to_rgba(c, alpha=0.3) for c in polygon_colors],
                        edgecolors='gray',
                        linewidths=1,
                        zorder=5
                    )
                    ax_main.add_collection(voronoi_collection)
                
                # Convert to percentages
                pitch_area = 105 * 68  # Standard pitch area
                total_area = sum(team_control.values())
                
                # If total area doesn't match pitch area, normalize
                scale_factor = pitch_area / total_area if total_area > 0 else 1
                
                team_pct = {
                    team_id: (area * scale_factor / pitch_area * 100) 
                    for team_id, area in team_control.items()
                }
                
                return team_pct
                
            except Exception as e:
                print(f"Error creating Voronoi diagram: {e}")
                import traceback
                traceback.print_exc()
                return {}
        
        def animate(frame_idx):
            """Animate one frame"""
            nonlocal voronoi_collection
            
            frame_id = unique_frames[frame_idx]
            frame_timestamp = frame_timestamps[frame_id]
            
            # Update title
            title.set_text(f"Game: {game_id} - Time: {frame_timestamp} (Frame {frame_idx+1}/{len(unique_frames)})")
            
            # Get current frame data
            frame_data = filtered_tracking[filtered_tracking['frame_id'] == frame_id]
            
            # Debug - print number of players in this frame
            players_by_team = frame_data[frame_data['player_name'] != 'Ball'].groupby('team_id').size()
            if frame_idx == 0:  # Only print for first frame to avoid spam
                print("\nPlayers by team in first frame:")
                for team_id, count in players_by_team.items():
                    team_name = team_name_map.get(team_id, f"Team {team_id}")
                    print(f"  - {team_name}: {count} players")
            
            # Update Voronoi diagram if enabled
            team_control = {}
            if show_voronoi:
                team_control = update_voronoi_diagram(frame_data)
            
                # Update control text box
                control_text_str = "Team Field Control:\n"
                for team_id, percentage in team_control.items():
                    # Only show teams with non-zero control
                    if percentage > 0:
                        team_name = team_name_map.get(team_id, f"Team {team_id}")
                        control_text_str += f"{team_name}: {percentage:.1f}%\n"
                control_text.set_text(control_text_str)
                
                # Update control percentage bar chart
                ax_control.clear()
                
                # Sort teams by control percentage
                teams_sorted = sorted(team_control.items(), key=lambda x: x[1], reverse=True)
                if teams_sorted:
                    team_ids = [team_id for team_id, _ in teams_sorted if _ > 0]
                    percentages = [pct for _, pct in teams_sorted if pct > 0]
                    if team_ids:
                        colors = [team_colors[team_id] for team_id in team_ids]
                        
                        # Create stacked horizontal bar chart
                        left = 0
                        for i, (team_id, pct) in enumerate(zip(team_ids, percentages)):
                            ax_control.barh(0, pct, left=left, color=colors[i], height=0.8)
                            # Add percentage text if space allows
                            if pct > 5:
                                ax_control.text(left + pct/2, 0, f"{pct:.1f}%", 
                                             ha='center', va='center', color='white', fontweight='bold')
                            left += pct
                
                # Remove axes ticks and labels
                ax_control.set_yticks([])
                ax_control.set_xticks([])
                ax_control.set_xlim(0, 100)
                ax_control.spines['top'].set_visible(False)
                ax_control.spines['right'].set_visible(False)
                ax_control.spines['bottom'].set_visible(False)
                ax_control.spines['left'].set_visible(False)
                ax_control.set_title("Team Field Control", fontsize=10, pad=2)
            
            # Clear existing player dots and labels
            for dot in player_dots.values():
                dot.remove()
            for label in player_labels.values():
                label.remove()
            for name in player_names.values():
                name.remove()
            player_dots.clear()
            player_labels.clear()
            player_names.clear()
            
            # Update player positions
            for _, player in frame_data.iterrows():
                player_id = player['player_id']
                x, y = player['x'], player['y']
                jersey_no = player['jersey_number'] if pd.notna(player['jersey_number']) else 0
                team_id = player['team_id']
                player_name = player['player_name']
                
                # Create player dots
                if player_name == 'Ball':
                    player_dots[player_id] = ax_main.scatter(x, y, s=100, color='yellow', 
                                                     edgecolor='black', zorder=20)
                else:
                    color = team_colors.get(team_id, 'gray')
                    player_dots[player_id] = ax_main.scatter(x, y, s=120, color=color, 
                                                     edgecolor='black', zorder=10)
                
                # Create jersey number labels
                if player_name != 'Ball':
                    player_labels[player_id] = ax_main.text(x, y, str(int(jersey_no)), fontsize=8, 
                                                    ha='center', va='center',
                                                    color='white', fontweight='bold', zorder=11)
                
                # Create player name labels
                if player_name != 'Ball':
                    player_names[player_id] = ax_main.text(x, y-2, player_name, fontsize=7, 
                                                   ha='center', color='white',
                                                   bbox=dict(facecolor='black', alpha=0.7, pad=1), 
                                                   zorder=12)
            
            # Return all artists that are being animated
            artists = [title, control_text, legend]
            artists.extend(list(player_dots.values()))
            artists.extend(list(player_labels.values()))
            artists.extend(list(player_names.values()))
            if voronoi_collection is not None:
                artists.append(voronoi_collection)
            
            return artists
        
        # Create animation
        if len(unique_frames) > 0:
            print(f"Creating animation with {len(unique_frames)} frames at {fps} fps...")
            
            # Create a variable to hold team_ids_for_points
            team_ids_for_points = []
            
            anim = animation.FuncAnimation(
                fig, animate, init_func=init,
                frames=len(unique_frames), interval=1000/fps, blit=True
            )
            
            print("Animation created successfully!")
            return anim
        else:
            print("No frames found for the specified time range.")
            return None
    
    finally:
        # Close the connection
        if conn:
            conn.close()
            print("Database connection closed.")


# Example usage:
game_id = '5oc8drrbruovbuiriyhdyiyok'
start_time = "00:00:00"
end_time = "00:00:30"

# Create animation with Voronoi diagrams and team control visualization
anim = create_football_animation(game_id, start_time, end_time, fps=2, show_voronoi=True)

# Display animation in notebook
from IPython.display import HTML
HTML(anim.to_jshtml())

HUYNA


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch
import matplotlib as mpl
from scipy.spatial import Voronoi, voronoi_plot_2d
from helperfunctions import get_database_connection, fetch_tracking_data

def visualize_football_field(game_id, frame_id=None, timestamp=None, show_voronoi=True):
    """
    Visualize the football field with player positions and Voronoi diagrams for a specific frame or timestamp.
    
    Parameters:
    - game_id (str): ID of the game to visualize
    - frame_id (int/str): Optional specific frame ID to visualize
    - timestamp (str): Optional timestamp to visualize (format: 'MM:SS' or 'HH:MM:SS')
    - show_voronoi (bool): Whether to display Voronoi diagrams (default: True)
    """
    # Connect to the database
    conn = get_database_connection()
    
    try:
        # Fetch tracking data for the specified game
        tracking_df = fetch_tracking_data(game_id, conn)
        print(f"Fetched {len(tracking_df)} tracking data points")
        
        # Check if we have any data
        if tracking_df.empty:
            print(f"No tracking data available for game ID: {game_id}")
            return
        
        # Fetch team names to match with team IDs
        team_names_query = """
        SELECT team_id, team_name FROM teams
        WHERE team_id IN (
            SELECT DISTINCT team_id FROM players
            WHERE player_id IN (
                SELECT DISTINCT player_id FROM player_tracking
                WHERE game_id = %s
            )
        )
        """
        team_names_df = pd.read_sql_query(team_names_query, conn, params=[game_id])
        
        # Create a mapping from team_id to team_name
        team_name_map = dict(zip(team_names_df['team_id'], team_names_df['team_name']))
        
        # Convert timestamp to appropriate format if provided and frame_id is not
        if timestamp and not frame_id:
            # Add hours if missing
            if len(timestamp.split(':')) == 2:
                timestamp = f"00:{timestamp}"
                
            # Find the closest frame to the specified timestamp
            tracking_df['timestamp_dt'] = pd.to_timedelta(tracking_df['timestamp'])
            target_dt = pd.to_timedelta(timestamp)
            
            # Find the closest frame
            closest_idx = abs(tracking_df['timestamp_dt'] - target_dt).idxmin()
            frame_id = tracking_df.loc[closest_idx, 'frame_id']
            
            print(f"Selected frame {frame_id} closest to timestamp {timestamp}")
        
        # If no frame or timestamp specified, list available frames
        if not frame_id:
            # Show available frames
            frame_samples = sorted(tracking_df['frame_id'].unique())
            if len(frame_samples) > 10:
                print("First 5 available frame IDs:")
                for i, frame in enumerate(frame_samples[:5]):
                    ts = tracking_df[tracking_df['frame_id'] == frame]['timestamp'].iloc[0]
                    print(f"{i+1}. Frame ID: {frame}, Timestamp: {ts}")
                print("...")
                print("Last 5 available frame IDs:")
                for i, frame in enumerate(frame_samples[-5:]):
                    ts = tracking_df[tracking_df['frame_id'] == frame]['timestamp'].iloc[0]
                    print(f"{len(frame_samples)-4+i}. Frame ID: {frame}, Timestamp: {ts}")
            else:
                print("All available frame IDs:")
                for i, frame in enumerate(frame_samples):
                    ts = tracking_df[tracking_df['frame_id'] == frame]['timestamp'].iloc[0]
                    print(f"{i+1}. Frame ID: {frame}, Timestamp: {ts}")
            
            # Use the first frame if none specified
            frame_id = tracking_df['frame_id'].min()
            print(f"\nNo frame specified. Using first frame: {frame_id}")
        
        # Filter data for the specific frame
        frame_data = tracking_df[tracking_df['frame_id'] == frame_id]
        
        # Check if we have data for the requested frame
        if frame_data.empty:
            print(f"No data found for frame {frame_id}")
            return
            
        # Get the timestamp for the selected frame
        selected_timestamp = frame_data['timestamp'].iloc[0] if not frame_data.empty else "Unknown"
        
        # Create a pitch
        pitch = Pitch(pitch_color='grass', line_color='white', pitch_type='opta',
                    pitch_length=105, pitch_width=68)
        fig, ax = pitch.draw(figsize=(12, 8))
        
        # Find unique teams and assign colors
        teams = frame_data['team_id'].unique()
        team_colors = {}
        
        # Assign colors to teams (excluding the ball if present)
        team_colors_list = list(mpl.colors.TABLEAU_COLORS.values())
        i = 0
        
        for team in teams:
            # Check if this is not the ball (assuming ball might have a specific team_id)
            team_data = frame_data[frame_data['team_id'] == team]
            if not ('Ball' in team_data['player_name'].values):
                team_colors[team] = team_colors_list[i % len(team_colors_list)]
                i += 1
                
        # Generate Voronoi diagram if requested
        if show_voronoi:
            # Filter out the ball and extract only player positions
            players_only = frame_data[frame_data['player_name'] != 'Ball']
            if len(players_only) >= 3:  # Need at least 3 points for a meaningful Voronoi diagram
                # Extract player positions
                points = players_only[['x', 'y']].values
                
                # Create Voronoi diagram
                try:
                    # Add boundary points to ensure Voronoi regions are contained within the pitch
                    pitch_boundary = np.array([
                        [0, 0], [105, 0], [105, 68], [0, 68],  # Corners
                        [0, 34], [105, 34],  # Middle of sidelines
                        [52.5, 0], [52.5, 68]  # Middle of goal lines
                    ])
                    
                    # Combine player positions with boundary points
                    all_points = np.vstack([points, pitch_boundary])
                    
                    # Calculate Voronoi diagram
                    vor = Voronoi(all_points)
                    
                    # Plot Voronoi diagram for players only (not boundary points)
                    for i, (_, player) in enumerate(players_only.iterrows()):
                        region_idx = vor.point_region[i]
                        region = vor.regions[region_idx]
                        
                        # Only plot finite regions
                        if -1 not in region and len(region) > 0:
                            polygon = [vor.vertices[j] for j in region]
                            polygon_array = np.array(polygon)
                            
                            # Get team color
                            team_id = player['team_id']
                            color = team_colors.get(team_id, 'gray')
                            
                            # Plot the Voronoi cell with transparency
                            ax.fill(polygon_array[:, 0], polygon_array[:, 1], 
                                   alpha=0.3, color=color, edgecolor='gray',
                                   linewidth=1, zorder=5)
                except Exception as e:
                    print(f"Error generating Voronoi diagram: {e}")
                    # Continue without Voronoi if it fails
        
        # Plot player positions
        for _, player in frame_data.iterrows():
            x = player['x']
            y = player['y']
            jersey_no = player['jersey_number']
            team_id = player['team_id']
            player_name = player['player_name']
            
            if player_name == 'Ball':
                # Plot the ball
                ax.scatter(x, y, s=100, color='yellow', edgecolor='black', zorder=20)
            else:
                # Plot the player
                color = team_colors.get(team_id, 'gray')  # Use gray for unassigned teams
                ax.scatter(x, y, s=120, color=color, edgecolor='black', zorder=10)
                
                # Add jersey number
                ax.text(x, y, str(jersey_no), fontsize=8, ha='center', va='center', 
                        color='white', fontweight='bold', zorder=11)
                
                # Add player name with small offset
                ax.text(x, y-2, player_name, fontsize=7, ha='center', color='white',
                        bbox=dict(facecolor='black', alpha=0.7, pad=1), zorder=12)
        
        # Add title with frame information
        title = f"Player Positions - Game: {game_id}\nFrame: {frame_id}, Time: {selected_timestamp}"
        if show_voronoi:
            title += " (with Voronoi Diagram)"
        ax.set_title(title, fontsize=14)
        
        # Add legend with team names
        handles = []
        labels = []
        for team_id, color in team_colors.items():
            # Use team name from the mapping if available, otherwise use team ID
            team_name = team_name_map.get(team_id, f"Team {team_id}")
            handles.append(plt.Line2D([0], [0], marker='o', color='w', 
                                     markerfacecolor=color, markersize=10))
            labels.append(team_name)
        
        # Add ball to legend if present
        ball_data = frame_data[frame_data['player_name'] == 'Ball']
        if not ball_data.empty:
            handles.append(plt.Line2D([0], [0], marker='o', color='w', 
                                    markerfacecolor='yellow', markersize=10))
            labels.append("Ball")
        
        if handles and labels:
            ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.05), 
                    fancybox=True, shadow=True, ncol=len(handles))
        
        plt.tight_layout()
        plt.show()
        
        return fig, ax
    
    finally:
        # Close the connection
        if conn:
            conn.close()
            print("Database connection closed.")

# Example usage - use one of these patterns:

# 1. Get information about available frames for a game
game_id = '5uts2s7fl98clqz8uymaazehg'
# visualize_football_field(game_id)

# 2. Visualize a specific frame
# visualize_football_field(game_id, frame_id=1722805622280)

# 3. Visualize at a specific timestamp with Voronoi diagram
visualize_football_field(game_id, timestamp="00:16:35", show_voronoi=True)

# 4. Visualize without Voronoi diagram
# visualize_football_field(game_id, timestamp="00:16:35", show_voronoi=False)